In [50]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm,trange
import torch.nn.functional as F
import csv
import os

# lyric data

In [22]:
lyrics = pd.read_csv('dataset/archive/lyrics-data.csv')
lyrics

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [23]:
lyrics = lyrics[lyrics['language'] == 'en']
lyrics

,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [24]:
lyrics.to_csv('dataset/enLirics.csv', index=False)

In [30]:
artists = pd.read_csv('dataset/archive/artists-data.csv')
artists

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
...,...,...,...,...,...
4163,Miriam Makeba,World Music; Black Music; Blues,17.0,0.0,/miriam-makeba/
4164,Freddie Aguilar,World Music,61.0,0.0,/freddie-aguilar/
4165,Amadou & Mariam,World Music,14.0,0.0,/amadou-mariam/
4166,Magic System,World Music; Gospel/Religioso,16.0,0.0,/magic-system/


In [31]:
# only keep the popular artists

artists = artists[(artists['Genres'].isin(['Rock'])) & (artists['Popularity']>2)]

In [32]:
artists.to_csv('dataset/popularArtists.csv', index=False)

In [33]:
df = lyrics.merge(artists[['Artist', 'Genres', 'Link']], left_on='ALink', right_on='Link', how='inner')

In [34]:
df

,ALink,SName,SLink,Lyric,language,Artist,Genres,Link
0,/4-non-blondes/,What's Up,/4-non-blondes/whats-up.html,Twenty-five years and my life is still\nTrying...,en,4 Non Blondes,Rock,/4-non-blondes/
1,/4-non-blondes/,Spaceman,/4-non-blondes/spaceman.html,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock,/4-non-blondes/
2,/4-non-blondes/,Pleasantly Blue,/4-non-blondes/pleasantly-blue.html,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock,/4-non-blondes/
3,/4-non-blondes/,I'm The One,/4-non-blondes/im-the-one.html,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock,/4-non-blondes/
4,/4-non-blondes/,Dear Mr. President,/4-non-blondes/dear-mr-president.html,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock,/4-non-blondes/
...,...,...,...,...,...,...,...,...
1298,/eagle-eye-cherry/,When Mermaids Cry,/eagle-eye-cherry/when-mermaids-cry.html,She was drowned in suicide\nFaithless lover ca...,en,Eagle-Eye Cherry,Rock,/eagle-eye-cherry/
1299,/eagle-eye-cherry/,While Away,/eagle-eye-cherry/while-away.html,When it's summer and you can't escape the heat...,en,Eagle-Eye Cherry,Rock,/eagle-eye-cherry/
1300,/eagle-eye-cherry/,Wishing It Was,/eagle-eye-cherry/wishing-it-was.html,Beauty and grace is what touches me most\nGood...,en,Eagle-Eye Cherry,Rock,/eagle-eye-cherry/
1301,/eagle-eye-cherry/,Worried Eyes,/eagle-eye-cherry/worried-eyes.html,I've tried so hard to remember what it is that...,en,Eagle-Eye Cherry,Rock,/eagle-eye-cherry/


In [35]:
# Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

In [36]:
len(df)

1247

In [37]:
#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 300)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [38]:
#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [39]:
test_set.head()

,index,ALink,SName,SLink,Lyric,language,Artist,Genres,Link,True_end_lyrics
0,928,/bruce-springsteen/,Whitetown,/bruce-springsteen/whitetown.html,Mary Ann on the avenue Girl all dressed up in ...,en,Bruce Springsteen,Rock,/bruce-springsteen/,"in Whitetown Hoo hoo, hoo hoo Oh oh oh Hoo hoo..."
1,402,/david-bowie/,Too Dizzy,/david-bowie/too-dizzy.html,There's too much talking for a night drive Too...,en,David Bowie,Rock,/david-bowie/,I'm helpless in love with you But you're just ...
2,706,/bruce-springsteen/,Lucky Town,/bruce-springsteen/lucky-town.html,House got too crowded clothes got too tight An...,en,Bruce Springsteen,Rock,/bruce-springsteen/,home Down in Lucky Town Down in Lucky Town I'm...
3,408,/david-bowie/,Uncle Arthur,/david-bowie/uncle-arthur.html,"Strikes the bell for 5 o'clock, Uncle Arthur c...",en,David Bowie,Rock,/david-bowie/,"money, he's well fed Uncle Arthur past the gas..."
4,512,/bruce-springsteen/,Bishop Dance,/bruce-springsteen/bishop-dance.html,Bishop danced with a thumbscrew woman Did a do...,en,Bruce Springsteen,Rock,/bruce-springsteen/,There's someone in the kitchen blowing Dinah o...


# Prepare the dataset

In [40]:
class SongLyrics(Dataset):
    
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
                
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [41]:
dataset = SongLyrics(df['Lyric'], truncate=True, gpt2_type="gpt2")

In [52]:
dataset.tokenizer("I have a pen, I have a apple!")

[40, 423, 257, 3112, 11, 314, 423, 257, 17180, 0]

In [46]:
dataset.tokenizer.convert_ids_to_tokens([40, 423, 257, 3112, 11, 314, 423, 257, 17180, 0])

['I', 'Ġhave', 'Ġa', 'Ġpen', ',', 'ĠI', 'Ġhave', 'Ġa', 'Ġapple', '!']

# Prepare training

In [47]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [48]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [49]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

# training loop

In [54]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=20, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            input_tensor = entry.to(device)
            # (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            # if carry_on and idx != len(train_dataloader) - 1:
            #     continue

            
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()



            # if (accumulating_batch_count % batch_size) == 0:
            #     optimizer.step()
            #     scheduler.step()
            #     optimizer.zero_grad()
            #     model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [55]:
#Train the model on the specific data we have

model = train(dataset, model, tokenizer)

/home/tian/mambaforge/envs/MyEnv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


947it [01:37,  9.72it/s]


Training epoch 1
tensor(1.3685, device='cuda:0', grad_fn=<NllLossBackward0>)


947it [01:30, 10.43it/s]


...

947it [01:30, 10.46it/s]


Training epoch 18
tensor(1.6027, device='cuda:0', grad_fn=<NllLossBackward0>)


947it [01:30, 10.46it/s]


Training epoch 19
tensor(1.4765, device='cuda:0', grad_fn=<NllLossBackward0>)


947it [01:30, 10.46it/s]


In [57]:
#Save the model to a pkl or something so it can be reused later on
torch.save(model, 'model/model.pt')

# Text generation

In [58]:
#Load the model to use it
model = torch.load('model/model.pt')

In [192]:
model.eval()
device=torch.device("cuda")
model = model.to(device)
with torch.no_grad():
    a = torch.tensor(tokenizer.encode("I have a dream that once I..")).unsqueeze(0)
    a = a.to(device)
    out = model(a, labels=a)
    out

In [193]:
out.logits.shape

torch.Size([1, 8, 50257])

In [215]:
_, lo = out[:2]
# the generated token's vector
lo_last = lo[:, -1, :]

In [217]:
sorted_logits, sorted_indices = torch.sort(lo_last, descending=True)
torch.sort(lo_last, descending=True)

torch.return_types.sort(
values=tensor([[-71.0671, -71.3038, -71.9985,  ..., -94.8994, -95.1810, -95.7999]],
       device='cuda:0'),
indices=tensor([[  198,   314,     1,  ..., 42889, 13945, 10298]], device='cuda:0'))

In [99]:
cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

tensor([[0.1640, 0.2780, 0.3837,  ..., 1.0000, 1.0000, 1.0000]],
       device='cuda:0')

In [114]:
sorted_indices_to_remove = cumulative_probs > 0.8
cumulative_probs > 0.8

tensor([[False, False, False,  ...,  True,  True,  True]], device='cuda:0')

In [107]:
sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
sorted_indices_to_remove

tensor([[False, False, False,  ...,  True,  True,  True]], device='cuda:0')

In [113]:
sorted_indices_to_remove[..., 0] = 0
sorted_indices_to_remove

tensor([[False, False, False,  ...,  True,  True,  True]], device='cuda:0')

In [125]:
indices_to_remove = sorted_indices[sorted_indices_to_remove]
lo_last[:, indices_to_remove] = -float("Inf")
lo_last

tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0')

In [133]:
torch.multinomial(F.softmax(lo_last, dim=-1), num_samples=1)

tensor([[13]], device='cuda:0')

In [157]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [163]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i, data in enumerate(test_data['Lyric']):
    x = generate(model=model.to('cpu'), tokenizer=tokenizer, prompt=data, entry_count=1, temperature=2.0)
    generated_lyrics.append(x)
  return generated_lyrics

In [151]:
test_set[:5]['Lyric'][4]

"Bishop danced with a thumbscrew woman Did a double-quick back-flip and slid across the floor The Catholic traffic flowed freely across the river And Fiddlestick fiddled quick out the front door Baby dumpling, mama's in the back tree If the bough breaks mama might fall Little sad and only, baby don't be lonely Mama knows 'rithmetic, knows how to take a fall Mama knows 'rithmetic, knows how to take a fall And the kids are crying Flapjacks, make'em fat, early in the morning Little Jack, grab your hat hear the breakfast call Muskrat, bad old cat, kick him in the fireplace There's someone in the kitchen blowing Dinah on their horn There's someone in the kitchen blowing Dinah like they're born Maverick Daddy got one-night prize The glides like a monkey-mule kickin' on the backslide Over hill, over hill, Daddy don't you spill now Papa got a switch-stick, he's pumping little Bill Papa got a switch-stick, he's pumping little Bill And Bill, he's crying Tomahawk, tomahawk, Daddy better duck out 

In [154]:
x = generate(model.to('cpu'), tokenizer, test_set[:5]['Lyric'][4], entry_count=1, temperature=8)
x

100%|██████████| 1/1 [00:20<00:00, 20.40s/it]


["Bishop danced with a thumbscrew woman Did a double-quick back-flip and slid across the floor The Catholic traffic flowed freely across the river And Fiddlestick fiddled quick out the front door Baby dumpling, mama's in the back tree If the bough breaks mama might fall Little sad and only, baby don't be lonely Mama knows 'rithmetic, knows how to take a fall Mama knows 'rithmetic, knows how to take a fall And the kids are crying Flapjacks, make'em fat, early in the morning Little Jack, grab your hat hear the breakfast call Muskrat, bad old cat, kick him in the fireplace There's someone in the kitchen blowing Dinah on their horn There's someone in the kitchen blowing Dinah like they're born Maverick Daddy got one-night prize The glides like a monkey-mule kickin' on the backslide Over hill, over hill, Daddy don't you spill now Papa got a switch-stick, he's pumping little Bill Papa got a switch-stick, he's pumping little Bill And Bill, he's crying Tomahawk, tomahawk, Daddy better duck out

In [164]:
generated_lyrics = text_generation(test_set[:5])

100%|██████████| 1/1 [00:19<00:00, 19.42s/it]


In [165]:
generated_lyrics

[['Mary Ann on the avenue Girl all dressed up in Kensington blue Down where the hammer falls She\'s born and bred within Whitetown walls Each and every thing slowly passes away She moves alone along the narrow street From a passing car she hears the music play Beneath the "El" she starts to sway Down in Whitetown, oh yeah Down in Whitetown, oh yeah Down in Whitetown Johnny\'s just trying to make his way He\'s got a job but mister it don\'t pay He sits and counts the scars On a stool down the end of a Whitetown bar And the old men here piss their lives away Drinking and joking about their cop-fighting days Ain\'t no answers here, just the past and fear Of spending the rest of your days Down in Whitetown, oh yeah Down in Whitetown, oh yeah Down in Whitetown Hoo hoo, hoo hoo Da da da Hoo hoo, hoo hoo Ah ah ah In a dream salvation comes with a beat Or the sound of breaking glass and running feet The beat that echoes down through the streets Mary Ann dances on Down in Whitetown, oh yeah Dow

In [167]:
test_set_sampled = test_set[:5]

In [172]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i, _ in enumerate(generated_lyrics):
  a = test_set_sampled ['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set_sampled['Generated_lyrics'] = my_generations

/tmp/ipykernel_27139/1906235433.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set_sampled['Generated_lyrics'] = my_generations


In [183]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set_sampled)):
  to_remove = test_set_sampled['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set_sampled['Generated_lyrics'][i].replace('',to_remove))

test_set_sampled['Generated_lyrics'] = final
test_set_sampled.head()

/tmp/ipykernel_27139/41216482.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set_sampled['Generated_lyrics'] = final


,index,ALink,SName,SLink,Lyric,language,Artist,Genres,Link,True_end_lyrics,Generated_lyrics
0,928,/bruce-springsteen/,Whitetown,/bruce-springsteen/whitetown.html,Mary Ann on the avenue Girl all dressed up in ...,en,Bruce Springsteen,Rock,/bruce-springsteen/,"in Whitetown Hoo hoo, hoo hoo Oh oh oh Hoo hoo...",map print turn from diploma roll See Shoes wa...
1,402,/david-bowie/,Too Dizzy,/david-bowie/too-dizzy.html,There's too much talking for a night drive Too...,en,David Bowie,Rock,/david-bowie/,I'm helpless in love with you But you're just ...,you're so burst breathOver June in desperatio...
2,706,/bruce-springsteen/,Lucky Town,/bruce-springsteen/lucky-town.html,House got too crowded clothes got too tight An...,en,Bruce Springsteen,Rock,/bruce-springsteen/,home Down in Lucky Town Down in Lucky Town I'm...,flagthere pony Albany romance land Montana pr...
3,408,/david-bowie/,Uncle Arthur,/david-bowie/uncle-arthur.html,"Strikes the bell for 5 o'clock, Uncle Arthur c...",en,David Bowie,Rock,/david-bowie/,"money, he's well fed Uncle Arthur past the gas...",book some cigarette vengeance Cain kneel descr...
4,512,/bruce-springsteen/,Bishop Dance,/bruce-springsteen/bishop-dance.html,Bishop danced with a thumbscrew woman Did a do...,en,Bruce Springsteen,Rock,/bruce-springsteen/,There's someone in the kitchen blowing Dinah o...,Cubs mute salute Stones separated rain kitten...


In [185]:
test_set_sampled['Generated_lyrics'][2]

' flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca nauseyon Tennaida dragonjam visit Mine<|endoftext|>  flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca nauseyon Tennaida dragonjam visit Mine<|endoftext|>f flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca nauseyon Tennaida dragonjam visit Mine<|endoftext|>l flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca nauseyon Tennaida dragonjam visit Mine<|endoftext|>a flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca nauseyon Tennaida dragonjam visit Mine<|endoftext|>g flagthere pony Albany romance land Montana precious phornout rock Californ Bed long paused farmer BarneyBack helps Gnacca n

In [186]:
test_set_sampled['True_end_lyrics'][2]

"home Down in Lucky Town Down in Lucky Town I'm gonna lose these blues I've found Down in Lucky Town"

# BLEU

In [188]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set_sampled)):
  reference = test_set_sampled['True_end_lyrics'][i]
  candidate = test_set_sampled['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/home/tian/mambaforge/envs/MyEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/tian/mambaforge/envs/MyEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/tian/mambaforge/envs/MyEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider usi

2.7456718009755223e-232

In [191]:
#Rouge score
from rouge import Rouge

rouge=Rouge()

rouge.get_scores(test_set_sampled['Generated_lyrics'], test_set_sampled['True_end_lyrics'], avg=True)

RecursionError: maximum recursion depth exceeded in comparison